In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import time

# GRAB ALL TICKERS INSIDE PRICE (MIN-MAX) RANGE

In [2]:
pages = []
tickers_list = []
tickers_group_list = []

ticker_skip = 1875
price_min = 5.00
price_max = 25.00
valid_page_status = True

start_time = time.time()
url = "" # previously outside while loop

while (valid_page_status):    
    
    url =   f"""
        https://www.marketwatch.com/tools/screener/stock?exchange=all&skip={ticker_skip}
        &orderbyfield=&direction=desc
        &visiblecolumns=Symbol,CompanyName,Price,NetChange,ChangePercent,Volume
        &pricemin={price_min}&pricemax={price_max}
        """ # must be inside while loop because {page_skip} won't update if outside of it...

    page = requests.get(url)
    soup_obj = soup(page.content, 'html.parser')
    ticker_skip += 25
    
    cells_found = soup_obj.find(class_="table__cell j-Symbol")

    if(cells_found):
        pages.append(url)
    else:
        valid_page_status = False
    
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Length of Pages[]: {len(pages)}")    
print(f"Elapsed Time: {elapsed_time}")

Length of Pages[]: 5
Elapsed Time: 9.022577285766602


In [3]:
for item in pages:
    page = requests.get(item)

    #soup_obj = soup(page.content, 'html.parser') 
    # https://stackoverflow.com/questions/49075563/typeerror-resultset-object-is-not-callable-python-with-beautifulsoup
    df = pd.read_html(page.content)

    tickers = df[0]
    tickers = tickers["Symbol"]
    
    # WHY MUST I ADD THEM INDIVIDUALLY VS. AS A GROUP ???
    for t in tickers:
        tickers_list.append(t)
        
    tickers_group_list.append(tickers)    
    
print(f"Size of Ticker List: {len(tickers_list)}")

Size of Ticker List: 105
['ANAB', 'AMX', 'AMTX', 'AMTB', 'AMSWA', 'AMSC', 'AMPY', 'AMPS', 'AMPL', 'AMLX', 'AMCX', 'AMCR', 'AMC', 'AMBI', 'AMBC', 'AMAM', 'AMAL', 'AM', 'ALVO', 'ALTU', 'ALTI', 'ALTG', 'ALRS', 'ALPN', 'ALOT', 'ALKT', 'ALIT', 'ALHC', 'ALEX', 'ALEC', 'ALDX', 'ALCO', 'ALCC', 'AKYA', 'AKR', 'AKO.B', 'AKO.A', 'AJX', 'AIV', 'AIRT', 'AIRS', 'AIP', 'AINC', 'AHH', 'AHCO', 'AGTI', 'AGS', 'AGRO', 'AGNC', 'AGL', 'AGI', 'AG', 'AFYA', 'AFRM', 'AFRI', 'AFCG', 'AFBI', 'AESI', 'AES', 'AEO', 'AEG', 'AEAE', 'ADTX', 'ADTN', 'ADT', 'ADSE', 'ADRT', 'ADPT', 'ADOC', 'ADEX', 'ADER', 'ADEA', 'ACVA', 'ACRV', 'ACRS', 'ACRE', 'ACR', 'ACOR', 'ACNT', 'ACMR', 'ACIW', 'ACIC', 'ACI', 'ACHR', 'ACEL', 'ACDC', 'ACCO', 'ACCD', 'ACBA', 'ACAX', 'ACAC', 'ACAB', 'ABR', 'ABOS', 'ABL', 'ABCM', 'ABCL', 'AAT', 'AAOI', 'AAN', 'AAMC', 'AAL', 'AADI', 'AACT', 'AAC']


# GRAB PRICE CHART TABLES FOR EVERY TICKER